In [ ]:
# Instalar YOLO y sus dependencias
!pip install tensorflow keras opencv-python matplotlib

## 1.2 Convertir las coordenadas del CSV en bounding boxes

Las coordenadas que tienes (Top_Left_X, Top_Left_Y, Bottom_Right_X, Bottom_Right_Y) definen rectángulos donde se encuentran los pingüinos en la imagen. Estas coordenadas serán las "bounding boxes" o cajas delimitadoras que tu modelo de detección de objetos debe aprender a predecir.

Tendrás que asegurarte de que estas coordenadas estén en el formato correcto (si ya están en píxeles, estás listo). Si estuvieran en un sistema de referencia espacial, puedes transformarlas a píxeles para que sean compatibles con las imágenes TIFF.

## 1.3 Generar etiquetas en formato YOLO o Faster R-CNN

Dependiendo del modelo que elijas, deberás convertir las coordenadas en el formato específico que requieren los frameworks de entrenamiento. Para YOLO, las etiquetas deberían estar en un formato como [class_id, center_x, center_y, width, height] normalizado entre 0 y 1. 

[class_id, x_center, y_center, width, height]


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from rasterio.transform import from_origin
import cv2

In [2]:
# Datos de entrada
CSV_COORDS = 'tiles_coords/coords_62.csv'
df = pd.read_csv(CSV_COORDS)
df.head()

,Id,X,Y
0,0,-59.231611,-62.301249
1,0,-59.231621,-62.301249
2,0,-59.231525,-62.301251
3,0,-59.231545,-62.301249
4,0,-59.231539,-62.301256


In [4]:
NUM_TILE = 62

if os.path.exists('/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'):
    path  = '/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'
else:
    path = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos'

image_name = f"recortes/recorte_{NUM_TILE}.tif"
tiff_file = os.path.join(path, image_name)


### Paso 2: Descargar el modelo YOLO preentrenado
Para este ejemplo, vamos a usar YOLOv5 de la biblioteca de Ultralytics, que es una de las implementaciones más populares y fáciles de usar. Puedes clonar su repositorio y seguir las instrucciones.

# Clonar el repositorio de YOLOv5
git clone https://github.com/ultralytics/yolov5.git
cd yolov5

# Instalar dependencias
pip install -r requirements.txt

2. Preparar los Datos
Los datos de entrenamiento deben estar en el formato adecuado para YOLO. Asegúrate de que tus coordenadas normalizadas estén guardadas en un archivo .txt para cada imagen.

Además, la estructura de directorios debería ser esta:

yolov5/
├── data/
│   ├── penguin_dataset.yaml   # Archivo de configuración de tu dataset
│   ├── images/
│   │   ├── train/              # Imágenes de entrenamiento
│   │   ├── val/                # Etiquetas de validación
│   └── labels/
│       ├── train/              # Etiquetas de entrenamiento
│       ├── val/                # Etiquetas de validación
├── yolov5/
└── train.py  

Cada archivo de etiqueta debe ser un archivo de texto .txt que tenga el mismo nombre que su imagen correspondiente. Por ejemplo, si tienes una imagen image1.jpg, el archivo de etiqueta debe llamarse image1.txt. y debe incluir dentro la siguiente información:

<class_id> <x_center> <y_center> <width> <height>

Estas etiquetas deben estar NORMALIZADAS, es decir, con valores entre 0 y 1.


Al final, deberías tener una estructura de directorios similar a esta:

data/
├── images/
│   ├── train/
│   │   ├── penguin1.jpg
│   │   ├── penguin2.jpg
│   │   └── penguin3.jpg
│   └── val/
│       ├── penguin_val1.jpg
│       └── penguin_val2.jpg
└── labels/
    ├── train/
    │   ├── penguin1.txt
    │   ├── penguin2.txt
    │   └── penguin3.txt
    └── val/
        ├── penguin_val1.txt
        └── penguin_val2.txt

Si tienes 100 imágenes de entrenamiento en la carpeta de IMAGES, tendrás 100 documentos de txt con su id, x, y, ancho y alto en la carpeta de LABELS.



### Paso 3: Cargar el modelo preentrenado
Una vez que tengas la biblioteca YOLOv5 instalada, puedes cargar un modelo preentrenado fácilmente. Aquí hay un ejemplo de cómo hacerlo en un script de Python:

In [ ]:
import torch

# Cargar el modelo preentrenado (puedes usar 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x')
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Establecer el modelo en modo de evaluación
model.eval()

### Paso 4: Realizar detección en imágenes
Una vez que tengas el modelo, puedes usarlo para hacer detección en imágenes. Aquí te muestro cómo hacerlo:

In [ ]:
import cv2
import numpy as np

# Cargar una imagen (asegúrate de tener la ruta correcta a la imagen)
image_path = 'ruta/a/tu/imagen.jpg'
image = cv2.imread(image_path)

# Realizar la detección
results = model(image)

# Mostrar los resultados
results.show()  # Muestra la imagen con bounding boxes


### Paso 5: Procesar los resultados
Los resultados de YOLO contienen información sobre las detecciones, incluyendo las coordenadas de las cajas delimitadoras, las clases y las puntuaciones de confianza. Aquí tienes un ejemplo de cómo acceder a esta información:

In [ ]:
# Obtener las detecciones
detections = results.xyxy[0]  # formato [x1, y1, x2, y2, conf, class]

for detection in detections:
    x1, y1, x2, y2, conf, cls = detection.numpy()
    print(f'Clase: {model.names[int(cls)]}, Confianza: {conf:.2f}, Coordenadas: [{x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f}]')

# Si quieres dibujar las detecciones en la imagen
for detection in detections:
    x1, y1, x2, y2, conf, cls = detection.numpy()
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Dibuja la caja
    cv2.putText(image, f'{model.names[int(cls)]} {conf:.2f}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Mostrar la imagen con detecciones
cv2.imshow('Detección', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Paso 6: Ajustar el modelo para tu caso específico (opcional)
Si quieres mejorar la precisión del modelo para detectar pingüinos específicamente, puedes hacer fine-tuning del modelo preentrenado usando tu propio conjunto de datos. Esto implicaría:

Preparar un conjunto de datos con tus imágenes y anotaciones.
Entrenar el modelo usando el conjunto de datos específico. Para esto, puedes seguir las instrucciones de entrenamiento en el repositorio de YOLOv5.
Conclusión
Estos pasos te permitirán empezar a trabajar con un modelo YOLO preentrenado utilizando Keras y PyTorch. Si decides hacer fine-tuning o necesitas más ayuda sobre cómo ajustar el modelo, no dudes en preguntar. ¡Buena suerte con tu proyecto!